In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader


In [21]:
!pip install torch torchvision matplotlib tqdm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 67.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 55.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 75.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [2]:

data = pd. read_csv("/content/drive/MyDrive/Neuro/combined_output.csv")

In [5]:
threshold = 2
train_df = data[data['folder_index'] <= threshold]
test_df = data[data['folder_index'] > threshold]

In [58]:
import torch
from torch.utils.data import Dataset

class FlyTrajectoryDataset(Dataset):
    def __init__(self, df, seq_len=5, target_len=1):
        self.seq_len = seq_len
        self.target_len = target_len
        self.samples = []

        # Group by each fly (folder_index)
        grouped = df.groupby('folder_index')

        for _, group in grouped:
            group = group.sort_values('frame').reset_index(drop=True)
            pos = group[['pos x', 'pos y']].values

            # Normalize using group mean and std
            mean = pos.mean(axis=0)
            std = pos.std(axis=0) + 1e-8  # avoid divide-by-zero
            norm_pos = (pos - mean) / std

            for i in range(len(norm_pos) - seq_len - target_len + 1):
                input_seq = norm_pos[i: i + seq_len]
                target_seq = norm_pos[i + seq_len: i + seq_len + target_len]
                self.samples.append((input_seq, target_seq))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        input_seq, target_seq = self.samples[idx]
        return torch.tensor(input_seq, dtype=torch.float32), torch.tensor(target_seq, dtype=torch.float32)


In [59]:
def create_dataloaders(df, folder_threshold, batch_size=32, seq_len=5, target_len=1):
    train_df = df[df['folder_index'] <= folder_threshold]
    test_df = df[df['folder_index'] > folder_threshold]

    train_dataset = FlyTrajectoryDataset(train_df, seq_len, target_len)
    test_dataset = FlyTrajectoryDataset(test_df, seq_len, target_len)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, test_loader


In [60]:
train_loader, test_loader = create_dataloaders(data, folder_threshold=2, batch_size=64, seq_len=5, target_len=1)


In [16]:
len(next(iter(train_loader)))

2

In [61]:
import torch
from torch.utils.data import Dataset

class FlySingleStepDataset(Dataset):
    def __init__(self, df):
        self.samples = []

        # Group data by individual fly/trajectory (folder_index)
        grouped = df.groupby('folder_index')

        for _, group in grouped:
            group = group.sort_values('frame').reset_index(drop=True)
            pos = group[['pos x', 'pos y']].values

            # Normalize the position using per-group mean and std
            mean = pos.mean(axis=0)
            std = pos.std(axis=0) + 1e-8  # avoid division by zero
            norm_pos = (pos - mean) / std

            # Make input → target pairs
            for i in range(len(norm_pos) - 1):
                self.samples.append((norm_pos[i], norm_pos[i + 1]))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        current_pos, next_pos = self.samples[idx]
        return torch.tensor(current_pos, dtype=torch.float32), torch.tensor(next_pos, dtype=torch.float32)


In [62]:
from torch.utils.data import DataLoader, random_split

def create_minimal_dataloaders(df, folder_threshold, batch_size=32, val_split=0.2):
    # Split by folder index
    train_df = df[df['folder_index'] <= folder_threshold]
    test_df = df[df['folder_index'] > folder_threshold]

    # Create full training dataset
    full_train_dataset = FlySingleStepDataset(train_df)
    test_dataset = FlySingleStepDataset(test_df)

    # Compute split sizes
    val_size = int(len(full_train_dataset) * val_split)
    train_size = len(full_train_dataset) - val_size

    # Split into train and val
    train_dataset, val_dataset = random_split(full_train_dataset, [train_size, val_size])

    # Create loaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, val_loader, test_loader, full_train_dataset



In [63]:
import pandas as pd

# Load your CSV
# df = pd.read_csv('fly_trajectory.csv')

# Create dataloaders
# train_loader, test_loader = create_minimal_dataloaders(data, folder_threshold=5, batch_size=64)
train_loader, val_loader, test_loader, train_dataset = create_minimal_dataloaders(data, folder_threshold=5, batch_size=64)

# Peek at a batch
for inputs, targets in train_loader:
    print("Input shape:", inputs.shape)   # [batch_size, 2]
    print("Target shape:", targets.shape) # [batch_size, 2]
    break


Input shape: torch.Size([64, 2])
Target shape: torch.Size([64, 2])


FORWARD ADDING NOISE

In [22]:
# Timesteps for diffusion
T = 1000

# Linear beta schedule
beta = torch.linspace(1e-4, 0.02, T)
alpha = 1. - beta
alpha_hat = torch.cumprod(alpha, dim=0)  # cumulative product



In [64]:
def q_sample(x_0, t, noise=None):
    """
    Sample from q(x_t | x_0) at timestep t
    """
    if noise is None:
        noise = torch.randn_like(x_0)
    sqrt_alpha_hat = torch.sqrt(alpha_hat[t])[:, None, None, None]
    sqrt_one_minus_alpha_hat = torch.sqrt(1 - alpha_hat[t])[:, None, None, None]
    return sqrt_alpha_hat * x_0 + sqrt_one_minus_alpha_hat * noise


In [78]:
import torch
import torch.nn.functional as F

def get_diffusion_params(T=1000, beta_start=1e-4, beta_end=0.02, eps=1e-8, device='cpu'):
    betas = torch.linspace(beta_start, beta_end, T, device=device)
    alphas = 1.0 - betas
    alphas_cumprod = torch.cumprod(alphas, dim=0)
    alphas_cumprod_prev = F.pad(alphas_cumprod[:-1], (1, 0), value=1.0)

    # Useful for reverse process
    sqrt_alphas = torch.sqrt(alphas + eps)
    sqrt_recip_alphas = torch.sqrt(1.0 / (alphas + eps))
    sqrt_alphas_cumprod = torch.sqrt(alphas_cumprod + eps)
    sqrt_one_minus_alphas_cumprod = torch.sqrt(1. - alphas_cumprod + eps)

    return {
        'T': T,
        'betas': betas,
        'alphas': alphas,
        'alphas_cumprod': alphas_cumprod,
        'alphas_cumprod_prev': alphas_cumprod_prev,
        'sqrt_alphas': sqrt_alphas,
        'sqrt_recip_alphas': sqrt_recip_alphas,
        'sqrt_alphas_cumprod': sqrt_alphas_cumprod,
        'sqrt_one_minus_alphas_cumprod': sqrt_one_minus_alphas_cumprod
    }


In [79]:
diffusion_params = get_diffusion_params(T=1000, device=device)

# Use in your functions like:
betas = diffusion_params['betas']
sqrt_alpha_hat = diffusion_params['sqrt_alphas_cumprod']


In [65]:
import torch
import numpy as np

def get_beta_schedule(T, beta_start=1e-4, beta_end=0.02):
    return torch.linspace(beta_start, beta_end, T)

T = 1000
betas = get_beta_schedule(T)
alphas = 1.0 - betas
alphas_bar = torch.cumprod(alphas, dim=0)

def forward_diffusion_sample(x_0, t, noise=None, eps=1e-8):
    """
    Perform the forward diffusion step: q(x_t | x_0)

    Args:
        x_0: Tensor [batch_size, 2] - original data (next position)
        t: Tensor [batch_size] - timestep indices (0 to T-1)
        noise: optional - if None, sampled from N(0,1)
        eps: float - small epsilon to prevent sqrt(0)

    Returns:
        x_t: noisy version of x_0
        noise: the noise added to x_0
    """
    if noise is None:
        noise = torch.randn_like(x_0)

    # Ensure t is long and on same device
    t = t.long()
    device = x_0.device

    sqrt_ab = torch.sqrt(alphas_bar[t].to(device) + eps)[:, None]
    sqrt_one_minus_ab = torch.sqrt(1. - alphas_bar[t].to(device) + eps)[:, None]

    x_t = sqrt_ab * x_0 + sqrt_one_minus_ab * noise

    return x_t, noise


In [80]:
def forward_diffusion_sample(x_0, t, diffusion_params, noise=None):
    """
    q(x_t | x_0) for any t ∈ [0, T)
    """
    if noise is None:
        noise = torch.randn_like(x_0)

    sqrt_alpha_bar_t = diffusion_params['sqrt_alphas_cumprod'][t].to(x_0.device).unsqueeze(-1)
    sqrt_one_minus_alpha_bar_t = diffusion_params['sqrt_one_minus_alphas_cumprod'][t].to(x_0.device).unsqueeze(-1)

    x_t = sqrt_alpha_bar_t * x_0 + sqrt_one_minus_alpha_bar_t * noise
    return x_t, noise


In [66]:
import torch.nn as nn

class DenoiseMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(2 + 1, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 2)
        )

    def forward(self, x_t, t):
        # Embed timestep and concat
        t = t.unsqueeze(1).float() / T
        x = torch.cat([x_t, t], dim=1)
        return self.net(x)


In [83]:
def get_loss(model, x_0, t):
    x_t, noise = forward_diffusion_sample(x_0, t)
    noise_pred = model(x_t, t)
    return torch.nn.functional.mse_loss(noise_pred, noise)


In [68]:
@torch.no_grad()
def sample(model, current_pos, steps=T):
    x = torch.randn(current_pos.shape).to(current_pos.device)
    for t in reversed(range(1, steps)):
        t_batch = torch.full((x.shape[0],), t, dtype=torch.long).to(x.device)
        z = torch.randn_like(x) if t > 1 else 0
        beta_t = betas[t]
        alpha_t = alphas[t]
        alpha_bar_t = alphas_bar[t]

        noise_pred = model(x, t_batch)

        coef1 = 1 / torch.sqrt(alpha_t)
        coef2 = (1 - alpha_t) / torch.sqrt(1 - alpha_bar_t)

        x = coef1 * (x - coef2 * noise_pred) + torch.sqrt(beta_t) * z

    return x  # This is x_0, i.e. predicted next position


In [70]:
import torch.optim as optim

# Define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = DenoiseMLP().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(50):
    for current_pos, next_pos in train_loader:
        next_pos = next_pos.to(device)
        t = torch.randint(0, T, (next_pos.size(0),), device=device)

        loss = get_loss(model, next_pos, t)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch}: loss = {loss.item():.4f}")

Epoch 0: loss = nan
Epoch 1: loss = nan


KeyboardInterrupt: 

In [71]:
class ConditionalDenoiseMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(2 + 2 + 1, 64),  # x_t + current_pos + timestep
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 2)
        )

    def forward(self, x_t, current_pos, t):
        t_embed = t.unsqueeze(1).float() / T
        inp = torch.cat([x_t, current_pos, t_embed], dim=1)
        return self.net(inp)


In [72]:
def get_conditional_loss(model, x_0, current_pos, t):
    x_t, noise = forward_diffusion_sample(x_0, t)
    noise_pred = model(x_t, current_pos, t)
    return torch.nn.functional.mse_loss(noise_pred, noise)


In [73]:
@torch.no_grad()
def sample_given_current_pos(model, current_pos, steps=T):
    x = torch.randn(current_pos.shape).to(current_pos.device)  # Start from noise
    for t in reversed(range(1, steps)):
        t_batch = torch.full((x.shape[0],), t, dtype=torch.long).to(x.device)
        z = torch.randn_like(x) if t > 1 else 0
        beta_t = betas[t]
        alpha_t = alphas[t]
        alpha_bar_t = alphas_bar[t]

        noise_pred = model(x, current_pos, t_batch)

        coef1 = 1 / torch.sqrt(alpha_t)
        coef2 = (1 - alpha_t) / torch.sqrt(1 - alpha_bar_t)

        x = coef1 * (x - coef2 * noise_pred) + torch.sqrt(beta_t) * z

    return x  # Final prediction of x_0 (next position)


In [81]:
def train_with_val(model, train_loader, val_loader, optimizer, diffusion_params, epochs=100, device='cuda', patience=5):
    model.to(device)
    best_val_loss = float('inf')
    epochs_no_improve = 0

    for epoch in range(epochs):
        model.train()
        total_loss = 0

        for current_pos, next_pos in train_loader:
            current_pos = current_pos.to(device)
            next_pos = next_pos.to(device)

            t = torch.randint(0, diffusion_params['T'], (next_pos.size(0),), device=device).long()

            # Forward diffusion
            x_t, noise = forward_diffusion_sample(next_pos, t, diffusion_params)

            # Predict the noise
            noise_pred = model(x_t, current_pos, t)

            loss = torch.nn.functional.mse_loss(noise_pred, noise)

            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()

            total_loss += loss.item()

        avg_train_loss = total_loss / len(train_loader)

        # Validation
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for current_pos, next_pos in val_loader:
                current_pos = current_pos.to(device)
                next_pos = next_pos.to(device)

                t = torch.randint(0, diffusion_params['T'], (next_pos.size(0),), device=device).long()
                x_t, noise = forward_diffusion_sample(next_pos, t, diffusion_params)
                noise_pred = model(x_t, current_pos, t)

                val_loss += torch.nn.functional.mse_loss(noise_pred, noise).item()

        avg_val_loss = val_loss / len(val_loader)

        print(f"Epoch {epoch+1}/{epochs} | Train Loss: {avg_train_loss:.6f} | Val Loss: {avg_val_loss:.6f}")

        # Early stopping
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1

        if epochs_no_improve >= patience:
            print(f"🛑 Early stopping at epoch {epoch+1}")
            break


In [75]:
@torch.no_grad()
def evaluate(model, test_loader, device='cuda'):
    model.eval()
    all_preds = []
    all_targets = []

    for current_pos, next_pos in test_loader:
        current_pos = current_pos.to(device)
        next_pos = next_pos.to(device)

        pred_next = sample_given_current_pos(model, current_pos)

        all_preds.append(pred_next.cpu())
        all_targets.append(next_pos.cpu())

    preds = torch.cat(all_preds, dim=0)
    targets = torch.cat(all_targets, dim=0)

    mse = torch.mean((preds - targets) ** 2).item()
    print(f"Test MSE: {mse:.4f}")

    return preds.numpy(), targets.numpy()


In [85]:
# train_loader, val_loader, test_loader, train_dataset = create_minimal_dataloaders(df, folder_threshold=5)

model = DenoiseMLP().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

train_with_val(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer,
    epochs=100,
    patience=5,
    device=device
)


TypeError: train_with_val() missing 1 required positional argument: 'diffusion_params'